In [1]:
import duckdb  # Importa o DuckDB para manipulação de dados e execução de SQL
import os  # Importa o módulo os para interagir com variáveis de ambiente do sistema
from dotenv import load_dotenv, find_dotenv  # Importa funções para carregar variáveis de ambiente de um arquivo .env
from datetime import datetime
import pandas as pd
from deltalake import DeltaTable, write_deltalake
# Carrega as variáveis de ambiente definidas no arquivo .env
load_dotenv(find_dotenv())

# Define as credenciais de acesso ao MinIO a partir das variáveis de ambiente
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_MINIO")  # Chave de acesso do MinIO
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY_MINIO")  # Chave secreta do MinIO
HOST_MINIO = os.getenv("HOST_MINIO")  # Host do MinIO

# Conecta ao DuckDB, criando uma instância de conexão
con = duckdb.connect()

# Cria uma secret no DuckDB para acessar o MinIO com as credenciais fornecidas
con.execute(f"""
    CREATE SECRET my_minio_secret (
        TYPE 'S3',
        KEY_ID '{AWS_ACCESS_KEY}',
        SECRET '{AWS_SECRET_KEY}',
        REGION 'us-east-1',
        ENDPOINT '{HOST_MINIO}:9000',
        URL_STYLE 'path',
        USE_SSL false
    );
""")

# Define o caminho de destino no MinIO para os arquivos Parquet
path_minio_landing = 's3://landing/comex'
path_minio_bronze = 's3://bronze/comex/pais_bloco'

In [2]:
storage_options = {
    "AWS_ENDPOINT_URL": f"http://{HOST_MINIO}:9000",
    "AWS_REGION": "us-east-1",
    "AWS_ACCESS_KEY_ID": AWS_ACCESS_KEY,
    "AWS_SECRET_ACCESS_KEY": AWS_SECRET_KEY,
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true",
    "AWS_ALLOW_HTTP": "true"
}

In [3]:
df = con.sql(f"""
        SELECT 
        CAST(CO_PAIS as int) AS cod_pais,
        CO_BLOCO as cod_bloco,
        NO_BLOCO as nome_bloco
         from '{path_minio_landing}/PAIS_BLOCO.parquet';        

        """).to_arrow_table()

In [4]:
df

pyarrow.Table
cod_pais: int32
cod_bloco: int64
nome_bloco: string
----
cod_pais: [[788,525,229,281,438,...,367,393,603,427,467]]
cod_bloco: [[51,51,51,51,51,...,22,22,22,22,22]]
nome_bloco: [["África","África","África","África","África",...,"União Europeia - UE","União Europeia - UE","União Europeia - UE","União Europeia - UE","União Europeia - UE"]]

In [5]:
write_deltalake(
        f'{path_minio_bronze}',
        df,
        storage_options=storage_options,
        
    )

In [5]:
table_path = f'{path_minio_bronze}'

# Conecte à tabela Delta existente
table = DeltaTable(table_path, storage_options=storage_options)



In [6]:
table.merge(
    source=df,
    predicate='target.cod_pais = source.cod_pais',
    source_alias="source",
    target_alias="target",
).when_matched_update(
    {
        "cod_bloco": "source.cod_bloco",
        "nome_bloco": "source.nome_bloco"
    }
).when_not_matched_insert_all().execute()


{'num_source_rows': 322,
 'num_target_rows_inserted': 178,
 'num_target_rows_updated': 144,
 'num_target_rows_deleted': 0,
 'num_target_rows_copied': 0,
 'num_output_rows': 322,
 'num_target_files_added': 2,
 'num_target_files_removed': 1,
 'execution_time_ms': 69,
 'scan_time_ms': 0,
 'rewrite_time_ms': 39}

In [7]:
con.sql(f"""
        SELECT *
FROM delta_scan('{path_minio_bronze}')
        """)

┌──────────┬───────────┬─────────────────────────────────┐
│ cod_pais │ cod_bloco │           nome_bloco            │
│  int32   │   int64   │             varchar             │
├──────────┼───────────┼─────────────────────────────────┤
│       40 │        51 │ África                          │
│      240 │        51 │ África                          │
│      623 │        51 │ África                          │
│      660 │        51 │ África                          │
│      329 │        51 │ África                          │
│       65 │       105 │ América Central e Caribe        │
│       41 │       105 │ América Central e Caribe        │
│       83 │       105 │ América Central e Caribe        │
│      698 │       105 │ América Central e Caribe        │
│       99 │       105 │ América Central e Caribe        │
│        · │        ·  │   ·                             │
│        · │        ·  │   ·                             │
│        · │        ·  │   ·                            

In [8]:
con.close()